nngraph
--------
nngraph makes it easier to express complex neural network architectures compared to just using containers like Sequential, Parallel, etc.

In [2]:
require 'nngraph';

The first type of new module you'll encounter is the Identity module. This module just takes in whatever's input and passes it on the next layer.

In [2]:
a = torch.Tensor{1,2,3}

Important note: The below code isn't nngraph yet. However, the Identity module is important for nngraph.

In [3]:
module1 = nn.Identity()

In [4]:
module1:forward(a)

 1
 2
 3
[torch.DoubleTensor of size 3]



Here is how this would be written in nngraph:

In [5]:
-- Notice the extra (). The extra () contain properties of this module when embedded into a graph.
x1 = nn.Identity()()
m = nn.gModule({x1},{x1})

In [6]:
m:forward(a)

 1
 2
 3
[torch.DoubleTensor of size 3]



gModule is the master module that indicates the input and output nodes of the graph. The above module's input and output node are both x1.

Let's try something more complex than this.  
x = a + b

In [7]:
-- Declare some tensors
t1 = torch.Tensor{1,2,3}
t2 = torch.Tensor{3,4,5}

Without nngraph.

In [8]:
-- nn.CAddTable adds tensors together. Obviously, dimensions need to match.
a = nn.CAddTable()

In [9]:
a:forward({t1,t2})

 4
 6
 8
[torch.DoubleTensor of size 3]



With nngraph.

In [10]:
a = nn.Identity()()
b = nn.Identity()()
x = nn.CAddTable()({a,b})
m = nn.gModule({a,b},{x})

In [11]:
m:forward({t1,t2})

 4
 6
 8
[torch.DoubleTensor of size 3]



CSubTable, CMulTable
--------------------

Element-wise subtraction and multiplication

In [12]:
a = nn.Identity()()
b = nn.Identity()()
x = nn.CSubTable()({a,b})
m = nn.gModule({a,b},{x})

In [13]:
m:forward({t1,t2})

-2
-2
-2
[torch.DoubleTensor of size 3]



In [15]:
a = nn.Identity()()
b = nn.Identity()()
x = nn.CMulTable()({a,b})
m = nn.gModule({a,b},{x})

In [16]:
m:forward({t1,t2})

  3
  8
 15
[torch.DoubleTensor of size 3]



SelectTable(index)
----------

Select one element out of a table.

In [23]:
k = {5, torch.Tensor{1,2,3}}

In [24]:
k

{
  1 : 5
  2 : DoubleTensor - size: 3
}


In [28]:
a = nn.Identity()()
x= nn.SelectTable(1)(a)
m = nn.gModule({a},{x})

In [29]:
m:forward(k)

5	


Negative index

In [30]:
a = nn.Identity()()
x= nn.SelectTable(-1)(a)
m = nn.gModule({a},{x})

In [31]:
m:forward(k)

 1
 2
 3
[torch.DoubleTensor of size 3]



Narrow (dim, offset, size)
=========

narrows down dimension dim from offset upto size.

In [3]:
nn.Narrow(1,2,3):forward(torch.Tensor(5,2):fill(1))

 1  1
 1  1
 1  1
[torch.DoubleTensor of size 3x2]



In [13]:
torch.Tensor(5,2):fill(1)

 1  1
 1  1
 1  1
 1  1
 1  1
[torch.DoubleTensor of size 5x2]



LookupTable(vocab_size, word_vec_size)
=====================

Creates a vector of size word_vec_size for each number from 1 to vocab_size.

In [8]:
m = nn.LookupTable(4, 5)

In [10]:
m:forward(torch.Tensor{4})

-2.1842 -1.4252 -0.9114 -0.2486 -0.6203
[torch.DoubleTensor of size 1x5]



In [12]:
m.weight

-0.5103 -0.6574 -1.5985  0.0040  1.6154
-1.2965 -0.3198 -0.8144 -0.2472 -0.1307
-0.1266 -1.5090 -0.9670  0.8578  0.3929
-2.1842 -1.4252 -0.9114 -0.2486 -0.6203
[torch.DoubleTensor of size 4x5]



ConcatTable()
==============

Insert modules in this module to make them parallel.

In [21]:
m = nn.ConcatTable()

In [22]:
m:add(nn.Linear(5,2));

In [23]:
m:add(nn.Linear(5,3));

In [24]:
m:forward(torch.randn(5))

{
  1 : DoubleTensor - size: 2
  2 : DoubleTensor - size: 3
}


SplitTable(dim)
=========

split a tensor into table of tensors along a given dimension

In [26]:
m = nn.SplitTable(1)

In [27]:
m:forward(torch.rand(3,2))

{
  1 : DoubleTensor - size: 2
  2 : DoubleTensor - size: 2
  3 : DoubleTensor - size: 2
}
